In [1]:
import os
import re
import json
import pandas as pd
import networkx as nx

## 1. Repositiories of Data

In [2]:
os.chdir(r'C:\Users\JABEERAK\GNN_CodeMapping\GCNCodeMap\Data')
print("Current Working Directory:", os.getcwd())

Current Working Directory: C:\Users\JABEERAK\GNN_CodeMapping\GCNCodeMap\Data


In [3]:
labels_dict = {'jabref': r'jabref_labels.txt',
            'sweetHome': r'sweethome3d_labels.txt',
            'ant': r'ant_labels.txt',
            'prom': r'prom_labels.txt',
            'teammates': r'teammates_labels.txt',
            'argouml': r'argouml_labels.txt',                   
            'lucene': r'lucene_labels.txt',
            'common': r'commons-imaging_labels.txt',        
}

roots = {
    'jabref': 'net/sf/jabref/',
    'sweetHome': 'com/eteks/sweethome3d/',
    'ant': 'org/apache/tools/',
    'prom': 'org/processmining/',
    'teammates': 'teammates/',
    'argouml': 'org/argouml/',
    'lucene': 'org/apache/lucene/',
    'common': 'org/apache/commons/imaging/'
}

In [15]:
def load_labels(labels_path):
    """Loads and parses the labels file into a structured dictionary."""
    with open(labels_path, 'r') as file:
        content = file.read()

    lines = content.split('\n')
    data = {
        'mapping': [],
        'relations': [],
        'roots': [],
        'modules': [] 
    }

    current_section = None
    for line in lines:
        line = line.strip()
        if not line or line.startswith('#'):
            if line.strip().startswith('# root-packages'):
                current_section = 'roots'
            elif line.strip().startswith('# mapping'):
                current_section = 'mapping'
            elif line.strip().startswith('# relations'):
                current_section = 'relations'
            elif line.strip().startswith('# modules'): 
                current_section = 'modules'
            continue

        if current_section == 'mapping':
            map_file = line.split()
            if len(map_file) == 2:
                data['mapping'].append({'Module': map_file[0], 'Entity': map_file[1]})
        elif current_section == 'relations':
            source_target = line.split()
            if len(source_target) == 2:
                data['relations'].append({'Source': source_target[0], 'Target': source_target[1]})
        elif current_section == 'roots':
            
            if '/' in line: 
                data['roots'].append(line.strip())
        elif current_section == 'modules':
            data['modules'].append(line.strip())

    labels = pd.DataFrame(data['mapping'])
    labels.Entity = labels.Entity.str.replace('..', '.')
    labels.Entity = labels.Entity.str.replace('\\.', '/')
    labels.Entity = labels.Entity.str.replace('*', '.*')
    labels['Entity'] = labels['Entity'].str.replace(r"\(\?:\?!", r"(?!", regex=True )
    
    architecture = pd.DataFrame(data['relations'])
    roots = data['roots']
    if len(roots) == 1:
        root = roots[0].strip('/')
    else:
        root = ''

    return architecture, labels, root


architecture, labels, root = load_labels(labels_dict['jabref'])

def read_and_flatten_json(file_path, encoding='utf-8'):
    """
    Reads a newline-delimited JSON file, flattens it, and returns the flattened DataFrame.
    """
    flattened_data_list = []
    try:
        with open(file_path, encoding=encoding) as f:
            for line in f:
                try:
                    data = json.loads(line)
                    flattened_data_list.append(pd.json_normalize(data))
                except json.JSONDecodeError as e:
                    print(f"Error decoding line: {line} -> {e}")
        flattened_data = pd.concat(flattened_data_list, ignore_index=True)

    except UnicodeDecodeError:
        with open(file_path, encoding='latin-1') as f:
            for line in f:
                try:
                    data = json.loads(line)
                    flattened_data_list.append(pd.json_normalize(data))
                except json.JSONDecodeError as e:
                    print(f"Error decoding line: {line} -> {e}")

        flattened_data = pd.concat(flattened_data_list, ignore_index=True)
        
    return flattened_data

df = read_and_flatten_json('jabref.json')

def cleaning(df, root):
    df=df.copy()
    df.rename(columns={'name': 'Entity'}, inplace=True)
    df.Entity = df.Entity.apply(lambda x: x.replace('.','/'))
    df = df[df.Entity.str.contains(root)]
    df.loc[:, 'Entity'] = df['Entity'].apply(lambda x: x.split('$')[0])
    df = df[~df.Entity.str.contains('package-info')]
    df['File'] = df.Entity.apply(lambda x: x.split(root)[1])
    df['File'] = df['File'].str.lstrip('/')
    return df

architecture, labels, root = load_labels(labels_dict['jabref'])

df = cleaning(df, root)

def extract_and_filter_dependencies(df):
    deps_list = []
    entity_set = set(df['Entity'])
    
    for deps in df['deps']:
        if isinstance(deps, list):
            deps_list.extend(deps)

    # Create DataFrame from dependencies
    deps_df = pd.DataFrame(deps_list)
    deps_df['source'] = deps_df['source'].apply(lambda x: x.replace('.','/'))
    deps_df['target'] = deps_df['target'].apply(lambda x: x.replace('.','/'))

    deps_df = deps_df[deps_df['source'].isin(entity_set) & deps_df['target'].isin(entity_set)]
    deps_df.drop_duplicates(inplace=True)
    deps_df.rename(columns={'source': 'Source', 'target': 'Target', 'type': 'Dependency_Type', 'count': 'Dependency_Count'}, inplace=True)
    
    return deps_df

df_dep = extract_and_filter_dependencies(df)


def clean_and_merge_entities(df):
    # Step 1: Clean the 'texts' column
    def clean_text_tokens(texts):
        cleaned_texts = []
        for token in texts:
            if re.fullmatch(r'[\n\t]{1,}', token):  # Matches tokens like [\n], [\n\n], or [\t]
                continue
            if len(token) == 1:
                continue
            if token.isdigit() or not any(char.isalpha() for char in token):
                continue
            if token in ['<init>', '<clinit>', '<p>']:
                continue
            token = token.replace(':', '')
            token = token.replace(',', '')
            token = token.replace('$', '')
            cleaned_texts.append(token)

        return cleaned_texts

    df['texts'] = df['texts'].apply(lambda x: clean_text_tokens(x) if isinstance(x, list) else [])

    merged_df = (
        df.groupby('Entity', as_index=False)
        .agg({
            'texts': lambda x: list(set(sum(x, []))),  # Deduplicate and merge text lists
            **{col: 'first' for col in df.columns if col not in ['Entity', 'texts']}
        })
    )

    return merged_df

df = clean_and_merge_entities(df)

def get_module(df, labels):
    df=df.copy()
    df['Module'] = None
    for _, row in labels.iterrows():
        file_pattern = row['Entity']
        module = row['Module']
        df.loc[df['Entity'].str.contains(file_pattern, regex=True, na=False), 'Module'] = module
        
    dff = df.copy()
    dff['File_ID'] = range(1, len(dff) + 1)
    dff.rename(columns={'texts': 'Code'}, inplace=True)
    dff = dff[~dff.Module.isna()]
    file_id_map = dict(zip(dff['Entity'], dff['File_ID']))
    dff = dff[['File_ID','File', 'Entity',	'Code','Module']]
    return dff, file_id_map

dff, file_id_map =  get_module(df,labels)

def get_dependencies(file_id_map,df, df_dep):
    df_dep = df_dep.copy()
    df = df.copy()

    df_merged_source = pd.merge(df_dep, df[['Entity', 'Module']], left_on='Source', right_on='Entity', how='left')
    df_merged_source = df_merged_source.rename(columns={'Module': 'Source_Module'}).drop(columns=['Entity'])

    df_merged_target = pd.merge(df_merged_source, df[['Entity', 'Module']], left_on='Target', right_on='Entity', how='left')
    df_merged_target = df_merged_target.rename(columns={'Module': 'Target_Module'}).drop(columns=['Entity'])

    df_dep = df_merged_target.copy()
    df_dep = df_dep[(~df_dep.Source_Module.isna()) & (~df_dep.Target_Module.isna())]
    df_dep['Source_ID'] = df_dep['Source'].map(file_id_map)
    df_dep['Target_ID'] = df_dep['Target'].map(file_id_map)

    df_dep = df_dep[['Source_ID', 'Source', 'Source_Module','Target_ID', 'Target', 'Target_Module', 'Dependency_Type', 'Dependency_Count']]
    allowed_set = set(zip(architecture['Source'], architecture['Target']))
    df_dep['Allowed'] = df_dep.apply(
        lambda row: 1 if (row['Source_Module'], row['Target_Module']) in allowed_set or row['Source_Module'] == row['Target_Module']
        else 0,
        axis=1
    )
    df_dep = df_dep[df_dep.Source != df_dep.Target]
    df_dep.drop_duplicates(inplace=True)
    return df_dep

df_dep = get_dependencies(file_id_map,dff, df_dep)

def generate_Graph(df, dep):
    modules = {file: df['Module'].iloc[i] for i, file in enumerate(df['File_ID'])}
    nodes = {file: df['Entity'].iloc[i] for i, file in enumerate(df['File_ID'])}
    G = nx.MultiDiGraph()
    for file in nodes.keys():
        G.add_node(file, code=nodes[file], label=modules[file])
    for _, row in dep.iterrows():
        if row['Source_ID'] in G.nodes and row['Target_ID'] in G.nodes:
            G.add_edge(row['Source_ID'], row['Target_ID'], type=row['Dependency_Type'], weight=row['Dependency_Count'])
            
    df['Closeness_Centrality'] = df['File_ID'].map(nx.closeness_centrality(G))
    return df

dff = generate_Graph(dff,df_dep)

In [16]:
dff

,File_ID,File,Entity,Code,Module,Closeness_Centrality
0,1,Globals,net/sf/jabref/Globals,"[IMPORT_FORMAT_READER, streamEavesdropper, get...",globals,0.126800
1,2,JabRefException,net/sf/jabref/JabRefException,"[getLocalizedMessage, Lnet/sf/jabref/JabRefExc...",model,0.123622
2,3,JabRefExecutorService,net/sf/jabref/JabRefExecutorService,"[command, execute, x1, - low prio, LOGGER, Ja...",logic,0.101804
3,4,JabRefGUI,net/sf/jabref/JabRefGUI,"[java.runtime.name, argsDatabases, </html>, wi...",gui,0.096597
4,5,JabRefMain,net/sf/jabref/JabRefMain,"[preferences, numericFields, proxyPreferences,...",gui,0.000000
...,...,...,...,...,...,...
1014,1015,shared/exception/SharedEntryNotPresentException,net/sf/jabref/shared/exception/SharedEntryNotP...,"[getNonPresentBibEntry, nonPresentBibEntry, Re...",logic,0.000000
1015,1016,shared/listener/OracleNotificationListener,net/sf/jabref/shared/listener/OracleNotificati...,"[dbmsSynchronizer, event, onDatabaseChangeNoti...",logic,0.066093
1016,1017,shared/listener/PostgresSQLNotificationListener,net/sf/jabref/shared/listener/PostgresSQLNotif...,"[dbmsSynchronizer, payload, channel, notificat...",logic,0.066062
1017,1018,shared/prefs/SharedDatabasePreferences,net/sf/jabref/shared/prefs/SharedDatabasePrefe...,"[password, rememberPassword, setType, sharedDa...",preferences,0.096625


## Making py File

In [6]:
import pandas as pd
import json
import re
import sys
import os
import networkx as nx


def load_labels(labels_path):
    """Loads and parses the labels file into a structured dictionary."""
    with open(labels_path, 'r') as file:
        content = file.read()

    lines = content.split('\n')
    data = {
        'mapping': [],
        'relations': [],
        'roots': [],
        'modules': [] 
    }

    current_section = None
    for line in lines:
        line = line.strip()
        if not line or line.startswith('#'):
            if line.strip().startswith('# root-packages'):
                current_section = 'roots'
            elif line.strip().startswith('# mapping'):
                current_section = 'mapping'
            elif line.strip().startswith('# relations'):
                current_section = 'relations'
            elif line.strip().startswith('# modules'): 
                current_section = 'modules'
            continue

        if current_section == 'mapping':
            map_file = line.split()
            if len(map_file) == 2:
                data['mapping'].append({'Module': map_file[0], 'Entity': map_file[1]})
        elif current_section == 'relations':
            source_target = line.split()
            if len(source_target) == 2:
                data['relations'].append({'Source': source_target[0], 'Target': source_target[1]})
        elif current_section == 'roots':
            
            if '/' in line: 
                data['roots'].append(line.strip())
        elif current_section == 'modules':
            data['modules'].append(line.strip())

    labels = pd.DataFrame(data['mapping'])
    labels.Entity = labels.Entity.str.replace('..', '.')
    labels.Entity = labels.Entity.str.replace('\\.', '/')
    labels.Entity = labels.Entity.str.replace('*', '.*')
    labels['Entity'] = labels['Entity'].str.replace(r"\(\?:\?!", r"(?!", regex=True )
    
    architecture = pd.DataFrame(data['relations'])
    roots = data['roots']
    if len(roots) == 1:
        root = roots[0].strip('/')
    else:
        root = ''

    return architecture, labels, root


def read_and_process_json(file_path, root):
    """Reads and cleans the JSON file, extracting relevant dependencies and entities."""
    flattened_data = []
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                try:
                    data = json.loads(line)
                    flattened_data.append(pd.json_normalize(data))
                except json.JSONDecodeError as e:
                    print(f"Error decoding line: {line} -> {e}")
    except UnicodeDecodeError:
        with open(file_path, encoding='latin-1') as f:
            for line in f:
                try:
                    data = json.loads(line)
                    flattened_data.append(pd.json_normalize(data))
                except json.JSONDecodeError as e:
                    print(f"Error decoding line: {line} -> {e}")

    df = pd.concat(flattened_data, ignore_index=True)
    df = df.rename(columns={'name': 'Entity'}).copy()
    df['Entity'] = df['Entity'].str.replace('.', '/')
    df = df[df['Entity'].str.contains(root)]
    df['Entity'] = df['Entity'].str.split('$').str[0]
    df = df[~df['Entity'].str.contains('package-info')]
    df['File'] = df['Entity'].str.split(root).str[1]
    return df


def clean_and_merge(df, labels):
    """Cleans and merges entities, then assigns modules."""
    def clean_tokens(texts):
        return [token for token in texts if len(token) > 1 and token.isalnum()]

    df['texts'] = df['texts'].apply(lambda x: clean_tokens(x) if isinstance(x, list) else [])
    df['Module'] = None
    for _, row in labels.iterrows():
        df.loc[df['Entity'].str.contains(row['Entity'], regex=True, na=False), 'Module'] = row['Module']

    df['File_ID'] = range(1, len(df) + 1)
    df = df[~df['Module'].isna()]
    return df


def extract_dependencies(df, df_dep, architecture):
    """Extracts and filters dependencies based on modules and architecture."""
    entity_set = set(df['Entity'])
    df_dep = pd.DataFrame(df_dep)
    df_dep['source'] = df_dep['source'].str.replace('.', '/')
    df_dep['target'] = df_dep['target'].str.replace('.', '/')
    df_dep = df_dep[df_dep['source'].isin(entity_set) & df_dep['target'].isin(entity_set)]
    df_dep = pd.merge(df_dep, df[['Entity', 'Module']], left_on='source', right_on='Entity', how='left').rename(columns={'Module': 'Source_Module'})
    df_dep = pd.merge(df_dep, df[['Entity', 'Module']], left_on='target', right_on='Entity', how='left').rename(columns={'Module': 'Target_Module'})
    allowed_set = set(zip(architecture['Source'], architecture['Target']))
    df_dep['Allowed'] = df_dep.apply(lambda row: 1 if (row['Source_Module'], row['Target_Module']) in allowed_set or row['Source_Module'] == row['Target_Module'] else 0, axis=1)
    return df_dep

def generate_Graph(df, dep):
    """Generates a graph and computes closeness centrality."""
    modules = {file: df['Module'].iloc[i] for i, file in enumerate(df['File_ID'])}
    nodes = {file: df['Entity'].iloc[i] for i, file in enumerate(df['File_ID'])}
    G = nx.MultiDiGraph()
    for file in nodes.keys():
        G.add_node(file, code=nodes[file], label=modules[file])
    for _, row in dep.iterrows():
        if row['Source_ID'] in G.nodes and row['Target_ID'] in G.nodes:
            G.add_edge(row['Source_ID'], row['Target_ID'], type=row['Dependency_Type'], weight=row['Dependency_Count'])
            
    df['Closeness_Centrality'] = df['File_ID'].map(nx.closeness_centrality(G))
    return df


def main(labels_path, json_path, dataset_name):
    """Main processing pipeline."""
    architecture, labels, root = load_labels(labels_path)
    df = read_and_process_json(json_path, root)
    df = clean_and_merge(df, labels)
    df_dep = extract_dependencies(df, df['deps'], architecture)
    df = generate_Graph(df, df_dep)

    # Ensure processed directory exists
    processed_dir = os.path.join("data", "processed")
    os.makedirs(processed_dir, exist_ok=True)

    # Save processed data in the processed directory
    df.to_csv(os.path.join(processed_dir, f'{dataset_name}_entities.csv'), index=False)
    df_dep.to_csv(os.path.join(processed_dir, f'{dataset_name}_dependencies.csv'), index=False)
    print(f"Processing complete. Results saved in '{processed_dir}'.")


if __name__ == "__main__":
    if len(sys.argv) < 4:
        print("Usage: python reprocess.py <labels_path> <json_path> <dataset_name>")
        sys.exit(1)
    labels_path = sys.argv[1]
    json_path = sys.argv[2]
    dataset_name = sys.argv[3]
    main(labels_path, json_path, dataset_name)


Usage: python reprocess.py <labels_path> <json_path> <dataset_name>


SystemExit: 1

In [20]:
import argparse

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Process JSON and labels files.")
    parser.add_argument('--labels', required=True, help="Path to the labels file")
    parser.add_argument('--json', required=True, help="Path to the JSON file")
    parser.add_argument('--name', required=True, help="name of the dataset")
    args = parser.parse_args()

    main(args.labels, args.json)


usage: ipykernel_launcher.py [-h] --labels LABELS --json JSON --name NAME
ipykernel_launcher.py: error: the following arguments are required: --labels, --json, --name


SystemExit: 2